In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from copy import deepcopy
from math import ceil, exp, atan, pi, cos, sin

from scipy.linalg import hankel
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import scipy.signal

import pylab as p
#import matplotlib.axes3d as p3
import mpl_toolkits.mplot3d.axes3d as p3

from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

ESPRIT simple

In [ ]:
from scipy.linalg import hankel

In [ ]:
def obtain_matrix(x, n): ### where is this function used ?
    X = hankel(x[:n], x[n-1:N])
    l = N -n+1  ################################ Roy: N = ??
    XH = X.conj().T
    Rxx = 1/l*X*XH
    U1, lam, U =  np.linalg.svd(Rxx)
    W = U1[:,0:K]
    return Rxx, W

In [ ]:
def esti_espace(Rxx, K):
    U1, lam, U2 = scipy.linalg.svd(Rxx)
    return U1[:,0:K]

In [ ]:
def esti_fre_amort(W):
    W_down = W[0:n-1,:]
    W_up = W[1:n,:]
    Phi = np.dot(np.linalg.pinv(W_down),(W_up))
    eig_values = scipy.linalg.eig(Phi, left = False, right = False)
    
    delta_vecteur = np.log(np.abs(eig_values))
    f_vecteur = 1/(2*np.pi)*np.angle(eig_values)

    return delta_vecteur, f_vecteur

In [ ]:
def LeastSquares(x,delta,f):
    N = len(x)
    t = np.array(range(N))
    k = delta+1j*2*np.pi*f
    tk = np.outer(t, k)
    alpha = np.dot(np.linalg.pinv(np.exp(tk)), x)
    a = abs(alpha)
    phi = np.angle(alpha)
    return a, phi, alpha

In [ ]:
def ESPRIT(x,n,K):
    # x: signal synthétique
    # N: longueur du signal
    # K: combien de sinus
    # n-K : diemension de l'espace bruit
    # n: le nombre de ligne n de la matrice espace signal
    
    N = len(x)
    l = N - n + 1
    X = hankel(x[0:n], x[n-1:N])
    # Construction de la matrice de corrélation
    Rxx = 1/l * X @ X.conj().T
    W = esti_espace(Rxx, K)
    delta, f = esti_fre_amort(W)
    a, phi, amplitude = LeastSquares(x,delta,f)
    return delta, f, a, phi, amplitude

adaptive computation of the spectral matrix

In [ ]:
#Programme permettant                                                                                                %
#  -Lire les donnees d'acceleration                                                                                  %
#  -Faire une FFT (du signal                                                                                          %
#  -Faire un calcul avec l'algorithme ESPRIT                                                                         %
#  -Export des resultats sous forme d'un fichier text lisible dans un tableur   

signal temporel

In [ ]:
def nextpow2(i):
    n = 1
    while n < i:
        n *= 2
    return n

In [ ]:
import scipy
from scipy.signal import lfilter

In [ ]:
#FIR 2
def filtre(signal,temps,fe,f_cen,f_delta,ordre):
    Nb = len(signal)
    ## calcul FFT
    Nfft = 2**nextpow(Nb)
    signal= np.fft.fft(signal, Nfft)
    freq = np.fft.fftfreq(Nfft, 1/fe)
    
    ## Translation du signal centré à fréquence nulle
    signal1 = np.multiply(signal, np.exp(-1j*2*np.pi*f_cen*temps))
    signal1_fft= np.fft.fft(signal1, Nfft)
    
    #FIR windowing
    cut_f = f_delta/2/(fe/2) # entre 0 et 1
    b = scipy.signal.firwin(ordre+1,cut_f)
    filtre = np.fft.fft(b, Nfft)
    #windowed signal2
    signal2 = lfilter(b, 1, signal1)
    signal2_fft = np.fft.fft(signal2, Nfft)
    return signal2, b

In [ ]:
def FIR_3(signal,temps,fe,f_cen,f_delta,ordre): 
    Nb = len(signal)
    ## calcul FFT
    Nfft = 2**nextpow(Nb)
    signal_FFT= np.fft.fft(signal, Nfft)
    axe_freq1 = np.arange(-fe/2, fe/2, fe/Nfft)
    
    freq_1 = (f_cen - f_delta/2)/(fe/2)
    freq_2 = (f_cen + f_delta/2)/(fe/2)
    b = scipy.signal.firwin(ordre+1,[freq_1, freq_2])
    filtre = np.fft.fft(b, Nfft)
    
    signal_filtre = lfilter(b, 1, signal)
    L = len(b)
    signal_fil_tron = signal_filtre[L:] # L ou L-1 
    Nfft2 = 2**nextpow(len(signal_fil_tron))
    signal_FFT2_tron= np.fft.fft(signal_fil_tron, Nfft2)
    axe_freq2 = np.arange(-fe/2, fe/2, fe/Nfft2)
    temps2 = temps[(L+1)/2: len(signal_fil_tron) + (L+1)/2]
    
    return signal_filtre, signal_fil_tron, b

In [ ]:
def deci(signal,temps,fe,R):
    fe_deci = fe/R
    t = np.arange(0, -1, R)
    signal_deci = signal[::R]
    temps_deci = temps[::R]
    Nb = len(signal) 
    N_deci = len(signal_deci)
    Nfft = 2**nextpow2(Nb)
    Nfft_deci = 2**nextpow2(N_deci)
    freq = np.arange(-fe/2, fe/2, fe/Nfft)
    freq_deci = np.arange(-fe_deci/2, fe_deci/2, fe_deci/Nfft_deci)
    return signal_deci, fe_deci

In [ ]:
def energie(delta, a, N):
    # Fonction permettant le calcul de l'énergie pour chaque composante extraite par la méthode ESPRIT. Le signal d'étude est de longueur 
    #N : longueur du signal d'étude
    E = []
    n = np.arange(0, N)
    for i in range(0, len(delta)):
        for j in n:
            E += a[i]**2*np.exp(-2*abs(delta[i])*n[j])
    Emax = max(E)
    EdB = 10*np.log(E/Emax)
    return E, EdB

In [ ]:
def ESTER(x, n, K):
    #x: signal
    # calcul de fonction d'erreur inverse
    N = len(x)
    l = N-n+1
    X = hankel(x[:n], x[n-1:N])
    XH = X.conj().T
    Rxx = 1/l*X*XH
    U1, lam, U =  np.linalg.svd(Rxx)
    norm_E = []
    for i in range(K):
        W = U1[: n, :i]
        W_down = W[0:n-1,:]
        W_up = W[1:n,:]
        Phi = np.dot(np.linalg.pinv(W_down),(W_up))
        E = W_up - np.dot(W_down, Phi)
        Valeur,_,_ = np.linalg.svd(E)
        norm_E = max(Valeur)  # not sure here
    return norm_E

In [ ]:
def synthesis(temps,delta,f,a,phi):
    Nb = len(delta)
    N = len(temps)
    signal = np.zeros(1, N)
    Te = temps[1] - temps[0]
    fe = 1/Te
    Nfft = 2**nextpow2(N)
    freq = np.arange(-fe/2, fe/2, fe/Nfft)
    for i in range(nb_composante):
        signal_ins = a[i]*np.multiply(np.exp(delta[i]*temps), np.cos(2*np.pi*temps*f[i]+ phi[i]))
        signal += signal_ins
    
    return signal

In [ ]:
# sachant déjà le signal, signal est le signal numérique à décimer
def program(signal, fe, t_i, t_f, f_cen, f_delta, ordre, R, n, K, nb_composante):
    #t_i: temps initial
    #t_f: temps final d'étude
    #ss : signal d'étude
    #f_cen : fréquence centrale
    #f_delta : 
    #R : step size
    Te = 1/fe
    ss = signal[np.floor(t_i*fe) : np.floor(t_f*fe)];
    temps = np.arange(0, Te**((np.floor(t_f*fe)-np.floor(t_i*fe))), Te)  #Decalage temporel du signal
    #Te = temps[1] - temps[0]
    #fe = 1/Te
    # (temps, ss), le signal temporel en représentation
    Nb = len(ss) # nombre de points du signal
    
    ## calcul FFT
    Nfft = 2**nextpow(Nb)
    signal= np.fft.fft(ss, Nfft)
    freq = np.fft.fftfreq(Nfft, 1/fe)
    # signal filtré
    s, b = filtre(ss, temps, fe, f_cen, f_delta, ordre)
    L_n = len(b)
    s = s[L_n:]
    retard = (L_n+1)/2*Te
    temps_filtre = temps[(L_n+1)/2: (len(s)+(L_n+1)/2)]
    
    #decimation du signal
    signal_deci, fe_deci = deci(signal,temps,fe,R)
    temps_deci = temps_filtre[::R]
    
    #ESPRIT 
    # n = np.floor(len(Signal_deci)/3)
    # K=50
    delta, f, a, phi, amp = ESPRIT(signal_deci, n, K)
    f, h = scipy.signal.freqz(b, 1) #H????
    f = f*fe_deci+f_cen
    delta = delta*fe_deci
    
    amp = 2*amp
    amp = np.multiply(amp, np.exp((abs(delta)-1j*2*np.pi*(f-f_cen))*retard))
    a = abs(amp)
    phi = np.angle(amp)
    
    EdB_sort = EdB.sort(axis=1)
    index = EdB.sort(axis=1)
    index = np.flip(index) 
    a = a[index]
    f = f[index]
    delta = delta[index]
    phi = phi[index]
    EdB = EdB[index]
    #np.argwhere(EdB>-40)
    print('frequency:', str(f[np.argwhere(EdB>-40)] ))
    print('damping', str(delta[np.argwhere(EdB>-40)] ))
    print('amplitude', str(a[np.argwhere(EdB>-40)] ))
    print('phase', str(phi[np.argwhere(EdB>-40)] ))
    
    norm_E = ESTER(signal_deci, n, K)
    # vous pouvez ajouter les figures
    signal_filtre, signal_fil_tron, b = FIR_3(signal,temps,fe,f_cen,f_delta,ordre)
    
    signal_syn = synthesis(temps, delta[: nb_composante], f[: nb_composante], a[: nb_composante], phi[: nb_composante])
    
    # il faut comparer signal_syn et signal_fil_tron